In [0]:
# INPE_Raw_Download — JOB 1 (RAW XML → Volume UC)
import re
import requests
from datetime import datetime
import os
import json

# ===============================
# PARAMETROS
# ===============================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("data_ref_carga", "")
dbutils.widgets.text("uuid_metadata", "")
dbutils.widgets.text("file_prefix", "")

catalog = dbutils.widgets.get("catalog").strip()
data_ref_carga = dbutils.widgets.get("data_ref_carga").strip()
uuid_metadata = dbutils.widgets.get("uuid_metadata").strip()
file_prefix = dbutils.widgets.get("file_prefix").strip()

# validações
if not catalog:
    raise ValueError("Parametro obrigatório: catalog")
if not data_ref_carga:
    raise ValueError("Parametro obrigatório: data_ref_carga (AAAA-MM-DD)")
if not uuid_metadata:
    raise ValueError("Parametro obrigatório: uuid_metadata")
if not file_prefix:
    raise ValueError("Parametro obrigatório: file_prefix")

In [0]:
# ===============================
# MONTAGEM DE NOMES E PATHS
# ===============================
file_prefix_clean = re.sub(r"[^0-9a-zA-Z_]", "_", file_prefix.lower())
data_yyyymmdd = data_ref_carga.replace("-", "")

file_name = f"{file_prefix_clean}_{data_yyyymmdd}.xml"
destino_base = f"/Volumes/{catalog}/raw/raw_tbra"
destino_final = f"{destino_base}/{file_name}"

# cria diretório (Volume UC) sem dbfs
os.makedirs(destino_base, exist_ok=True)

print(f"Destino final UC Volume: {destino_final}")

In [0]:
# ===============================
# DOWNLOAD DO XML
# ===============================
url_download = (
    f"https://terrabrasilis.dpi.inpe.br/geonetwork/srv/api/records/"
    f"{uuid_metadata}/formatters/xml"
)

print("Download URL:", url_download)

try:
    response = requests.get(url_download, timeout=90)
    response.raise_for_status()
    content = response.text
except Exception as e:
    raise Exception(f"Erro ao baixar o XML: {e}")

# valida XML mínimo
if not content.lstrip().startswith("<?xml") and "<gmd:MD_Metadata" not in content:
    raise Exception("Conteúdo baixado não parece XML válido.")

In [0]:
# ===============================
# GRAVAÇÃO NO VOLUME (SEM DBFS)
# ===============================
with open(destino_final, "w", encoding="utf-8") as f:
    f.write(content)

print(f"Arquivo salvo com sucesso em: {destino_final}")

In [0]:
# ===============================
# MANIFEST (TAMBÉM DIRETO EM VOLUME)
# ===============================
manifest = {
    "file_name": file_name,
    "path": destino_final,
    "uuid_metadata": uuid_metadata,
    "file_prefix": file_prefix_clean,
    "data_ref_carga": data_ref_carga,
    "download_ts": datetime.utcnow().isoformat() + "Z"
}

manifest_path = f"{destino_base}/_download_manifest_{data_yyyymmdd}.json"

with open(manifest_path, "w", encoding="utf-8") as mf:
    mf.write(json.dumps(manifest, ensure_ascii=False, indent=2))

print(f"Manifest salvo em: {manifest_path}")

dbutils.notebook.exit("SUCCESS")